# Introduction to TensorFlow: Model saving and restoring

#### Note: 
The way serialization is carried out in TF is via *checkpoint files* (usually .ckpt). The way this is done has changed quite a lot from version to version. Most chnages were mainly adding to the previous functionality and increase scope. This was true till version 0.12(the latest) when they changed their way of doing this 
more drastically and unfortunately not compatible with previous of Tensorflow.

Prior to r0.12, the saver would produce *two* files:
```
name_of_ckpt.ckpt
name_of_ckpt.ckpt.meta
```
And a checkpoint file << checkpoint >> (one per folder of ckpt files). You would explicilty save and restore << name_of_ckpt.ckpt >>. See screenshot below (for the models saved in this tutorial)

![Image](screenshot_ckpts.png)

From r0.12, we would get, by default, *three* similar files:
```
name_of_ckpt.ckpt.data00000-of-00001
name_of_ckpt.ckpt.meta
name_of_ckpt.ckpt.index
```    

#### Revert to old version
If you are using r0.12, use: 
```
saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)
```
to revert to the previous way of serializing.


#### More info:
For more information and (more advanced) functionality, check out saving and restoring documentation:
[r0.12](https://www.tensorflow.org/api_docs/python/state_ops/saving_and_restoring_variables)
[<r0.12](https://www.tensorflow.org/versions/r0.11/how_tos/variables/#saving_and_restoring)

In [1]:
# for compatibility 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
# import tensorflow
import tensorflow as tf

In [3]:
# let create some variables
A = tf.Variable(tf.random_normal([ 3, 3]), name="A")
b = tf.Variable(tf.random_normal([ 3, 1]), name="b")
prod = tf.matmul(A,b)

# update value of b
norm = tf.sqrt(tf.reduce_sum(tf.square(prod), 0))
update_op = b.assign(prod/norm)

In [4]:
# init operation
init_op = tf.initialize_all_variables()

## Saving a model
Now, we create a Saver with *tf.train.Saver()* to manage all variables in the model (default).

In [5]:
saver = tf.train.Saver()

We launch a session, where will update the value of vector b and will save the session current state at the end.

In [6]:
name_of_checkpoint = "model.ckpt"

In [7]:
with tf.Session() as sess:
    sess.run(init_op)
    
    print("Initial values:")
    print(sess.run(A))
    print(sess.run(b))
    
    print("\n Starting updating")
    for _ in range(10):
        sess.run(update_op)
        print(sess.run(b).T)
        
    print("\n Saving model to: " + name_of_checkpoint)
    saver.save(sess,name_of_checkpoint)

Initial values:
[[ 0.89158756 -2.03744626  0.67002648]
 [-1.93548453 -1.43480539  0.69013792]
 [ 2.41397119  1.11396372  0.91476625]]
[[-0.13989319]
 [-0.15827264]
 [ 0.63065863]]

 Starting updating
[[ 0.55274266  0.83146513  0.0560467 ]]
[[-0.34100622 -0.65175551  0.67744333]]
[[ 0.54685915  0.7633028  -0.3439678 ]]
[[-0.39416012 -0.72602606  0.5634926 ]]
[[ 0.51252121  0.74680007 -0.42380613]]
[[-0.42232612 -0.73781157  0.52656895]]
[[ 0.49173525  0.74429601 -0.45190692]]
[[-0.43859816 -0.74057829  0.50909287]]
[[ 0.47944552  0.74362183 -0.46600285]]
[[-0.4481349  -0.74158514  0.49922606]]

 Saving model to: model.ckpt


## Restoring a model
Can create a similar Saver, or even re-use the one above. I will create a new one.

In [8]:
saver2restore = tf.train.Saver()

Launch a session to restore and check the values. These should match the ones at the end of the session above

In [9]:
with tf.Session() as sess:
    sess.run(init_op)
    saver2restore.restore(sess, name_of_checkpoint)
    
    print("Restored values:")
    print(sess.run(A))
    print(sess.run(b))

Restored values:
[[ 0.89158756 -2.03744626  0.67002648]
 [-1.93548453 -1.43480539  0.69013792]
 [ 2.41397119  1.11396372  0.91476625]]
[[-0.4481349 ]
 [-0.74158514]
 [ 0.49922606]]


## Saving only certain variables/parameters rather then the whole session
This is useful, when computational graph is quite complex/large but we are interested only in some of the values -- like results of some computations. In this case, we specify when creating the Saver which variable(s) we are interested in. 

For instance in the example above, we might only be interested in the value of the computed eigenvector b.

In [10]:
saver = tf.train.Saver({"eigenv": b})
name_of_checkpoint = 'b.ckpt'

In [11]:
with tf.Session() as sess:
    sess.run(init_op)
    
    print("Initial values:")
    print(sess.run(A))
    print(sess.run(b))
    
    print("\n Starting updating")
    for _ in range(10):
        sess.run(update_op)
        print(sess.run(b).T)
        
    print("\n Saving model to: " + name_of_checkpoint)
    saver.save(sess, name_of_checkpoint)

Initial values:
[[ 0.49205759 -0.64811355 -0.74128866]
 [ 1.82169056  0.17344774 -0.69482893]
 [-0.11688004  0.55249608  0.6057356 ]]
[[ -9.25583899e-01]
 [ -4.07729030e-01]
 [  2.67610740e-04]]

 Starting updating
[[-0.10804801 -0.9919517  -0.06601086]]
[[ 0.69546574 -0.35174373 -0.62658101]]
[[ 0.50524825  0.80147791 -0.3199335 ]]
[[-0.02598014  0.98886055  0.14655995]]
[[-0.76659727  0.02247914  0.6417346 ]]
[[-0.41481304 -0.87912714  0.23466079]]
[[ 0.17002998 -0.95004958 -0.26171643]]
[[ 0.75517285  0.27623892 -0.59447956]]
[[ 0.32223347  0.93462145 -0.15049337]]
[[-0.3370398   0.85728496  0.38918725]]

 Saving model to: b.ckpt


## Restoring b
Please note that b gets restored correctly, while A - since it was not saved - will be re-intialize to a random value.

In [12]:
with tf.Session() as sess:
    sess.run(init_op)
    saver.restore(sess, name_of_checkpoint)
    
    print("Restored values:")
    print(sess.run(A))
    print(sess.run(b))

Restored values:
[[-0.3682647  -0.15525812  0.53426868]
 [-1.81205511  0.57222915 -0.46779445]
 [-0.05808004 -1.37002575 -1.73209405]]
[[-0.3370398 ]
 [ 0.85728496]
 [ 0.38918725]]
